In [1]:
using Pkg

In [85]:
using CSV, DataFrames, JuMP, Gurobi, PyCall, LinearAlgebra, Ipopt
using JuMP, Juniper, Ipopt, Statistics

In [13]:
function haversine_distance(coord1, coord2)
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = deg2rad(coord1[1]), deg2rad(coord1[2])
    lat2, lon2 = deg2rad(coord2[1]), deg2rad(coord2[2])

    # Calculate the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)^2 + cos(lat1) * cos(lat2) * sin(dlon / 2)^2
    c = 2 * atan(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c

    return distance
end

# Helper function to convert degrees to radians
function deg2rad(deg)
    return deg * π / 180
end

# Example usage:
coord1 = (37.7749, -122.4194)  # Latitude and longitude for San Francisco, CA
coord2 = (34.0522, -118.2437)  # Latitude and longitude for Los Angeles, CA

distance = haversine_distance(coord1, coord2)
println("The distance between the two locations is $distance km.")

The distance between the two locations is 559.1205770615527 km.


1:10000

In [83]:
function classic_distance_model()
    ipopt = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
    optimizer = optimizer_with_attributes(Juniper.Optimizer, "nl_solver"=>ipopt)
    model_a = Model(optimizer)
    facilities_index = 1:10 #we use i for this indexing
    clients_index = 1:1000 #we use j for this indexing
    @variable(model_a, x[facilities_index, clients_index], Bin)
    @variable(model_a, y[facilities_index], Bin)
    @constraint(model_a, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1)
    @constraint(model_a, [i in facilities_index, j in clients_index], x[i,j] <= y[i])
    # facilities_cost = sum(c[i]*y[i] for i in facilities_index) # need to find c
    facilities_cost = 0
    equity_cost = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in clients_index)
    @objective(model_a, Min, facilities_cost+equity_cost);
    return model_a
end

classic_distance_model (generic function with 1 method)

In [ ]:
# african_indices
# american_indices

$$d(P_i) := \frac{1}{|P_i|}\sum_{j \in P_i} d(j, \pi(j)),$$

$d(j, pi(j)) = \sum_{i} x_{i,j}d(i,j)$

In [86]:
# Example data: 8 lists containing indices of students in each major
major1 = [1, 2, 3, 4, 5]
major2 = [6, 7, 8, 9, 10]
major3 = [11, 12, 13, 14, 15]
major4 = [16, 17, 18, 19, 20]
major5 = [21, 22, 23, 24, 25]
major6 = [26, 27, 28, 29, 30]
major7 = [31, 32, 33, 34, 35]
major8 = [36, 37, 38, 39, 40]

# Combine the majors into an array of arrays
majors = [major1, major2, major3, major4, major5, major6, major7, major8]

# Example function f (you can replace this with your own function)
function f(student_index)
    # Example function: Square of the student index
    return student_index^2
end

# Calculate the average of the function across each major
averages = [mean(f.(major)) for major in majors]

# Print the results
println("Averages: ", averages)


Averages: [11.0, 66.0, 171.0, 326.0, 531.0, 786.0, 1091.0, 1446.0]


In [94]:
# Define the range
full_range = 1:1000

# Specify the number of sub-arrays
num_subarrays = 8

# Calculate the size of each sub-array
subarray_size = length(full_range) ÷ num_subarrays

# Split the range into sub-arrays
races = [full_range[(i-1)*subarray_size+1:i*subarray_size] for i in 1:num_subarrays]

8-element Vector{UnitRange{Int64}}:
 1:125
 126:250
 251:375
 376:500
 501:625
 626:750
 751:875
 876:1000

In [97]:
function lp_distance_model(p)

    #set_up
    ipopt = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
    optimizer = optimizer_with_attributes(Juniper.Optimizer, "nl_solver"=>ipopt)
    model_a = Model(optimizer)

    #indices
    facilities_index = 1:10 #we use i for this indexing
    clients_index = 1:1000 #we use j for this indexing


    #whether client is matched to facility
    @variable(model_a, x[facilities_index, clients_index], Bin)

    #whether facility is built/operated
    @variable(model_a, y[facilities_index], Bin)

    #every client gets matched to some service provider
    @constraint(model_a, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1)

    # you can only match client to an operating service provider
    @constraint(model_a, [i in facilities_index, j in clients_index], x[i,j] <= y[i])


    
    # facilities_cost = sum(c[i]*y[i] for i in facilities_index) 
    facilities_cost = 0

    equity_cost = 0
    for race in races
        value = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in race)/length(race)
        equity_cost += value^p
    end 
    @NLobjective(model_a, Min, facilities_cost+equity_cost);
    return model_a
end

lp_distance_model (generic function with 2 methods)

In [ ]:
optimize!(lp_distance_model(2))

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 10010
#IntBinVar: 10010
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 25005.882311105728
Relaxation Obj: 0.0008000272097948911

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                          0.0                -   59966.0     0         -     


[ Info: Breaking out of strong branching as the time limit of 100.0 seconds got reached.


    2       3                 -                          0.0                -   73916.9     -         >>    


In [71]:
@variable(model_a, x[facilities_index, clients_index], Bin);

In [72]:
@variable(model_a, y[facilities_index], Bin);

In [73]:
@constraint(model_a, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1);

In [74]:
@constraint(model_a, [i in facilities_index, j in clients_index], x[i,j] <= y[i]);

In [75]:
facilities_cost = 0

0

In [76]:
function d(i,j)
    return 1
end

d (generic function with 1 method)

In [79]:
# @NLexpression(model, obj_expr, sum(x[i] for i in 1:3))
# @NLobjective(model, Min, obj_expr)
equity_cost = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in clients_index)
@objective(model_a, Min, facilities_cost+equity_cost);

┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP ~/.julia/packages/JuMP/OUdu2/src/operators.jl:279


In [81]:
optimize!(model_a)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 100010
#IntBinVar: 100010
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 5.699100971221924
Relaxation Obj: 9.999999925032196

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                          10.0               -    100.8      0         -     


[ Info: Breaking out of strong branching as the time limit of 100.0 seconds got reached.


    2       3                 -                          10.0               -    110.3      -         >>    
    2       4                 -                          10.0               -    119.8      -         >>    
    2       5                 -                          10.0               -    129.4      -         >>    


LoadError: InterruptException:

In [51]:
using JuMP
using Ipopt

function create_model()
    model = Model(Ipopt.Optimizer)

    # Define decision variables
    @variable(model, x[facilities_index, clients_index],Bin)
    @variable(model, y[facilities_index], Bin)

    # # Add constraints
    # @constraint(model, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1)
    # @constraint(model, [i in facilities_index, j in clients_index], x[i,j] <= y[i])

    # # Set nonlinear objective function
    
    # # facilities_cost = sum(c[i]*y[i] for i in facilities_index) # need to find c
    # facilities_cost = 0
    # equity_cost = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in clients_index)
    # # @NLexpression(model, obj_expr, sum(x[i] for i in 1:3))
    # # @NLobjective(model, Min, obj_expr)
    # @objective(model, Min, facilities_cost+equity_cost)
    # return model
end

# Solve the model
model = create_model()
optimize!(model)

# Print results
println("Optimal solution:")
println("x = ", value.(x))
println("Objective value = ", objective_value(model))


LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.ZeroOne are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [58]:
# Define decision variables
model_2 = Model(Ipopt.Optimizer)
@variable(model_2, x[facilities_index, clients_index], binary=true)

LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.ZeroOne are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.